In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, StratifiedKFold, HalvingGridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, fbeta_score, f1_score, make_scorer, \
                            average_precision_score, precision_recall_curve, confusion_matrix,balanced_accuracy_score, cohen_kappa_score
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import pickle
import subprocess
import lime
import shap
from pdpbox import pdp

c:\Users\swamy\anaconda3\envs\bafd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Load Data**

In [6]:
# Read every column except 'device_fraud_count' as its value is a constant 0
df = pd.read_csv('../data/raw/Base.csv', usecols=lambda x: x != 'device_fraud_count')

### **Handle Missing Values**

In [8]:
# Features with missing values represented by negative values according to documentation
missing_features = ['prev_address_months_count', 'current_address_months_count', 'intended_balcon_amount',
                    'bank_months_count', 'session_length_in_minutes', 'device_distinct_emails_8w']

# Replace negative values with NaN
for feature in missing_features:
    df[feature] = df[feature].apply(lambda x: x if x >= 0 else np.nan)

### **Encode missing values**

In [9]:
features_to_drop = ['prev_address_months_count', 'intended_balcon_amount', 'bank_months_count']
for col in features_to_drop:
    missing_column_name = f'{col}_missing'
    df[missing_column_name] = np.where(df[col].isna(), 1, 0)

Drop features with a high percentage of missing values, and have very weak correlation with fraud status.

In [10]:
df.drop(features_to_drop, axis=1, inplace=True)

Drop rows with missing values as a very small percentage of the remaining observations have missing values.

In [11]:
df.dropna(inplace=True)

### **Handle Categorical Features**

Perform dummy encoding. Very similar to one-hot encoding, but the first encoded column is dropped to reduce correlation between encoded columns.

In [12]:
# Only features with String data type need to be encoded
encoded_features = [feature for feature in df.columns if df[feature].dtype == 'object']

df = pd.get_dummies(df, columns=encoded_features, drop_first=True, dtype=int)

### **Train-Test Split**

In [13]:
# Separate the feature matrix and target variable
X = df.drop('fraud_bool', axis=1)
y = df['fraud_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### **Feature Scaling**

#### Min-Max Scaling (Normalization)

From EDA, numerical features were identified. Min-max scaling is applied as parametric models are sensitive to scale.

In [14]:
numeric_features = ['income', 'name_email_similarity', 'current_address_months_count', 'customer_age', 'days_since_request', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
                    'velocity_4w', 'bank_branch_count_8w', 'date_of_birth_distinct_emails_4w', 'credit_risk_score', 'proposed_credit_limit', 'session_length_in_minutes']

scaler = MinMaxScaler()

# Fit only on the training data
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

### **Feature Selection - Backward Stepwise (logistic model)**

In [15]:
def backward_stepwise_selection(X, y, p_threshold=0.05):
    features = X.columns.tolist()
    num_features = len(features)

    for i in range(num_features, 0, -1):
        model = sm.Logit(y, X[features]).fit()
        p_values = model.pvalues
        max_p_value = p_values.max()
        if max_p_value > p_threshold:
            remove_feature = p_values.idxmax()
            print(f"Removing '{remove_feature}' with p-value: {max_p_value:.4f}")
            features.remove(remove_feature)
        else:
            break

    return features

selected_features = backward_stepwise_selection(X_train, y_train)

# New selected features
print(f"Number of features selected: {len(selected_features)}")
for i in range(0, len(selected_features), 5):
    print(selected_features[i:i+5])

         Current function value: 0.048577
         Iterations: 35
Removing 'housing_status_BG' with p-value: 0.9969
Optimization terminated successfully.
         Current function value: 0.048581
         Iterations 11
Removing 'device_distinct_emails_8w' with p-value: 0.7865
Optimization terminated successfully.
         Current function value: 0.048581
         Iterations 11
Removing 'employment_status_CG' with p-value: 0.7445
Optimization terminated successfully.
         Current function value: 0.048581
         Iterations 11
Removing 'payment_type_AC' with p-value: 0.6812
Optimization terminated successfully.
         Current function value: 0.048581
         Iterations 11
Removing 'payment_type_AE' with p-value: 0.1891
Optimization terminated successfully.
         Current function value: 0.048583
         Iterations 11
Removing 'bank_months_count_missing' with p-value: 0.1106
Optimization terminated successfully.
         Current function value: 0.048585
         Iterations 11
R

In [16]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]

### **Resampling**

Fraud class vs non fraud class

In [17]:
ratio = y.value_counts() / len(y) * 100
print(f'% of non-fraud class in y: {round(ratio[0],3)}%\n% of fraud class in y: {round(ratio[1],3)}%\n')

ratio_train = y_train.value_counts() / len(y_train) * 100
print(f'% of non-fraud class in y_train: {round(ratio_train[0],3)}%\n% of fraud class in y_train: {round(ratio_train[1],3)}%\n')

ratio_test = y_test.value_counts() / len(y_test) * 100
print(f'% of non-fraud class in y_test: {round(ratio_test[0],3)}%\n% of fraud class in y_test: {round(ratio_test[1],3)}%')

% of non-fraud class in y: 98.893%
% of fraud class in y: 1.107%

% of non-fraud class in y_train: 98.893%
% of fraud class in y_train: 1.107%

% of non-fraud class in y_test: 98.893%
% of fraud class in y_test: 1.107%


SMOTE

In [18]:
smote = SMOTE(random_state=42, sampling_strategy = 0.666) #ratio of minority:majority 40:60

Xt_resampled_SMOTE, yt_resampled_SMOTE = smote.fit_resample(X_train, y_train)

ratio_SMOTE = yt_resampled_SMOTE.value_counts() / len(yt_resampled_SMOTE) * 100
print(f'% of non-fraud class in resampled data: {round(ratio_SMOTE[0],3)}%\n% of fraud class in resampled data: {round(ratio_SMOTE[1],3)}%')

% of non-fraud class in resampled data: 60.024%
% of fraud class in resampled data: 39.976%


### **Evaluation metrics**

In [19]:
def evaluate_results(y_test, y_pred):
    score_results = {}
    score_results["accuracy_score"] = accuracy_score(y_test, y_pred)
    score_results["classification_report"] = classification_report(y_test, y_pred)
    score_results["recall_score"] = recall_score(y_test, y_pred)
    score_results["precision_score"] = precision_score(y_test, y_pred)
    score_results["F2-score"] = fbeta_score(y_test, y_pred, beta=2)
    score_results["F1-score"] = f1_score(y_test, y_pred)
    score_results["average_precision_score"] = average_precision_score(y_test, y_pred)
    score_results["PR-AUC"] = precision_recall_curve(y_test, y_pred)
    score_results["balanced_accuracy_score"] = balanced_accuracy_score(y_test, y_pred)
    score_results["Kappa statistics"] = cohen_kappa_score(y_test,y_pred)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    TN, FP, FN, TP = cm.ravel()
    TPR = TP/(TP+FN)
    FNR = FN/(TP+FN)
    score_results["TPR"] = TPR
    score_results["FNR"] = FNR
    return score_results

'''
## To use function
xgb_base_pred = xgb_base.predict(X_test_new)

xgb_base_results = evaluate_results(y_test=y_test_new, y_pred=xgb_base_pred)
'''

'\n## To use function\nxgb_base_pred = xgb_base.predict(X_test_new)\n\nxgb_base_results = evaluate_results(y_test=y_test_new, y_pred=xgb_base_pred)\n'

### **Models**

#### Decision Tree (Baseline)


In [ ]:
dt = DecisionTreeClassifier(random_state=42)
evaluate_results(dt,"Decision Tree",Xt_resampled_SMOTE, yt_resampled_SMOTE)

Decision Tree Model Accuracy on Train Data:1.0
Decision Tree Model Performance on Test Data:
Decision Tree Precision: 0.05956196581196581
Decision Tree Recall: 0.10140973169622555
Decision Tree F2: 0.08891547049441786
Decision Tree TPR: 0.10140973169622555
Decision Tree FNR: 0.8985902683037744
Decision Tree Classification Report: 
              precision    recall  f1-score   support

           0       0.99      0.98      0.99    196523
           1       0.06      0.10      0.08      2199

    accuracy                           0.97    198722
   macro avg       0.52      0.54      0.53    198722
weighted avg       0.98      0.97      0.98    198722



### **Save and load model**

In [41]:
def save_model(model, model_filename):
    pickle.dump(model, open(model_filename,"wb"))

# This function is more specific to the results from lazy classifier. feel free to overwrite it
def save_results(results_df, results_filename):
    results_df = results_df["evaluate_results"].reset_index()
    # Convert the 'Metrics' column into separate columns
    df_metrics = pd.json_normalize(results_df['evaluate_results'])

    # Concatenate the two DataFrames
    results_df = pd.concat([results_df['Model'].rename('Model'), df_metrics], axis=1)
    results_df.to_csv(results_filename, index=False)

def save_model_and_results(model, model_filename, results_df, results_filename):
    save_model(model, model_filename)
    save_results(results_df, results_filename)

def load_model(model_filename):
    return pickle.load(open(model_filename, "rb"))

'''
## Sample usage
save_model_and_results(clf_smote_encoded,"C:/NUS/Fraud-Hackathon/models/baseline_encoded.pkl", models_smote_encoded, "C:/NUS/Fraud-Hackathon/models/baseline_encoded_results.csv")
'''

'\n## Sample usage\nsave_model_and_results(clf_smote_encoded,"C:/NUS/Fraud-Hackathon/models/baseline_encoded.pkl", models_smote_encoded, "C:/NUS/Fraud-Hackathon/models/baseline_encoded_results.csv")\n'

In [42]:
save_model(dt, '../models/decision_tree.pkl')